In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import torchvision.models as models
from pycocotools.coco import COCO
import cv2
import numpy as np
import os
from util import gaussian_heatmap, visualize_sample

In [ ]:
ANNOTATION_PATH = "./dataset/annotation/aligned_gauge_keypoints.json"
IMAGE_PATH = "./dataset/image"

In [ ]:
class GaugeDataset(Dataset):
    def __init__(self, annotation_path, image_path, transform=None, sigma=2):
        self.coco = COCO(annotation_path)
        self.ids = list(self.coco.imgs.keys())

        # 어노테이션 데이터가 있는 이미지만 사용
        all_ids = list(self.coco.imgs.keys())
        self.ids = []
        for img_id in all_ids:
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            if len(ann_ids) > 0:
                self.ids.append(img_id)
        
        self.image_path = image_path
        self.transform = transform
        self.input_size = 640
        self.output_size = 160
        self.sigma = sigma
        self.img_info = None

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)[0]

        img_path = os.path.join(self.image_path, img_info["file_name"])
        image = cv2.imread(img_path)

        if image is None:
            raise FileNotFoundError(f"Image file not found: {img_path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        kp = np.array(anns["keypoints"]).reshape(-1, 3)

        heatmaps = gaussian_heatmap(
            keypoints=kp[:, :2],
            input_size=self.input_size,
            output_size=self.output_size,
            sigma=self.sigma
            )

        if self.transform:
            image = self.transform(image)

        return image, heatmaps

In [ ]:
dataset = GaugeDataset(annotation_path=ANNOTATION_PATH, image_path=IMAGE_PATH)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
